In [1]:
from praatio import textgrid
import soundfile as sf
from gudhi.point_cloud import timedelay
import numpy as np
from ripser import ripser

import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.io as pio
%matplotlib qt5

inputFile="E:\\MFA\\output\\ALL_049_F_ENG_ENG_HT1.Textgrid"
tg=textgrid.openTextgrid(inputFile,includeEmptyIntervals=False)
phoneTier=tg.tierDict['phones']

specified_vowel=['ɑ']

wavFile='E:\\MFA\\input\\ALL_049_F_ENG_ENG_HT1.wav'
sig,samplerate=sf.read(wavFile)
specified_vowel_list=[ele for ele in phoneTier.entryList if ele[2] in specified_vowel]

M=100
max_edge_length=1

# wav_fraction_finder is to find the corresponding wav signal according to interval
def wav_fraction_finder(start_time, end_time):
    sig_fraction=sig[int(start_time*samplerate):int(end_time*samplerate)]
    return sig_fraction

# head_tail_scissor is to erase signal in head and tail that has amplitude smaller than 0.05
# can also use it to see if the length of renewing signal is greater than 500 or not 
def head_tail_scissor(sig):
    valid_interval=[index for index in range(len(sig)) if (sig[index]>0.03)]
    if len(valid_interval)==0:
        return False,sig
    head=min(valid_interval)
    tail=max(valid_interval)
    sig=sig[head:tail+1]
    if tail-head<500:
        return False,sig
    return True,sig

# principle_frequency_finder is to find the principle frequency of a speech signal
def principle_frequency_finder(sig):
    t=int(len(sig)/2)
    corr=np.zeros(t)

    for index in np.arange(t):
        ACF_delay=sig[index:]
        L=(t-index)/2
        m = np.sum(sig[int(t-L):int(t+L+1)]**2) + np.sum(ACF_delay[int(t-L):int(t+L+1)]**2)
        r = np.sum(sig[int(t-L):int(t+L+1)]*ACF_delay[int(t-L):int(t+L+1)])
        corr[index] = 2*r/m

    zc = np.zeros(corr.size-1)
    zc[(corr[0:-1] < 0)*(corr[1::] > 0)] = 1
    zc[(corr[0:-1] > 0)*(corr[1::] < 0)] = -1

    admiss = np.zeros(corr.size)
    admiss[0:-1] = zc
    for i in range(1, corr.size):
        if admiss[i] == 0:
            admiss[i] = admiss[i-1]

    maxes = np.zeros(corr.size)
    maxes[1:-1] = (np.sign(corr[1:-1] - corr[0:-2])==1)*(np.sign(corr[1:-1] - corr[2::])==1)
    maxidx = np.arange(corr.size)
    maxidx = maxidx[maxes == 1]
    max_index = 0
    if len(corr[maxidx]) > 0:
        max_index = maxidx[np.argmax(corr[maxidx])]

    return (max_index, corr)
    
specified_valid_vowel_list=[head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[1] for ele in specified_vowel_list if head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[0]]
specified_duration_list=[ele[0] for ele in specified_vowel_list if head_tail_scissor(wav_fraction_finder(ele[0],ele[1]))[0]]


print('There are ',str(len(specified_valid_vowel_list)),' phones in the specified list.')

T=[0]*len(specified_valid_vowel_list)
for i in range(len(specified_valid_vowel_list)):
    T[i],corr=principle_frequency_finder(np.array(specified_valid_vowel_list[i]))
    T[i]=T[i]/samplerate

delay=[round(ele*samplerate*2*np.pi/M) for ele in T]
for element in range(len(delay)):
    if delay[element]==0:
        delay[element]=1

There are  23  phones in the specified list.


In [12]:
# # a mild change
# element=10
# data=specified_valid_vowel_list[element][410:1610]
# fig=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data))),
#     y=data,
#     ),
# )

# fig.update_layout(
#     template='plotly_white',
# )
# fig.show()

# point_Cloud=timedelay.TimeDelayEmbedding(M, 1, 1)
# Points=point_Cloud(data)
# print(len(Points))
# dgms = ripser(Points,maxdim=1)['dgms']
# dgms = dgms[1]
# x=[dgms[ele][0] for ele in range(len(dgms))]
# y=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]

# #pio.write_image(fig, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_mild_series.pdf',scale=1,width=1000)

# fig3=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data[0:600]))),
#     y=data[0:600],
#     ),
# )

# fig3.update_layout(
#     template='plotly_white',
# )
# fig3.show()
# #pio.write_image(fig3, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_mild_series1.pdf',scale=1,width=500)

# fig4=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data[0:800]))),
#     y=data[0:800],
#     ),
# )

# fig4.update_layout(
#     template='plotly_white',
# )
# fig4.show()
# #pio.write_image(fig4, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_mild_series2.pdf',scale=1,width=650)

# fig5=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data[0:1000]))),
#     y=data[0:1000],
#     ),
# )

# fig5.update_layout(
#     template='plotly_white',
# )
# fig5.show()
# #pio.write_image(fig5, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_mild_series3.pdf',scale=1,width=800)


In [3]:
# a mild change
element=10
data=specified_valid_vowel_list[element]

point_Cloud=timedelay.TimeDelayEmbedding(M, 1, 1)
Points=point_Cloud(data)
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
xm=[dgms[ele][0] for ele in range(len(dgms))]
ym=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]


Points=point_Cloud(data[0:600])
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
xm_1=[dgms[ele][0] for ele in range(len(dgms))]
ym_1=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]

Points=point_Cloud(data[0:800])
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
xm_2=[dgms[ele][0] for ele in range(len(dgms))]
ym_2=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]

Points=point_Cloud(data[0:1000])
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
xm_3=[dgms[ele][0] for ele in range(len(dgms))]
ym_3=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]


1767
501
701
901


In [9]:
# a rapid change
element=7
data=specified_valid_vowel_list[element]

point_Cloud=timedelay.TimeDelayEmbedding(M, 1, 1)
Points=point_Cloud(data)
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
x=[dgms[ele][0] for ele in range(len(dgms))]
y=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]


Points=point_Cloud(data[0:600])
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
x_1=[dgms[ele][0] for ele in range(len(dgms))]
y_1=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]

Points=point_Cloud(data[0:800])
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
x_2=[dgms[ele][0] for ele in range(len(dgms))]
y_2=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]

Points=point_Cloud(data[0:1000])
print(len(Points))
dgms = ripser(Points,maxdim=1)['dgms']
dgms = dgms[1]
x_3=[dgms[ele][0] for ele in range(len(dgms))]
y_3=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]


1977
501
701
901


In [11]:
# # a rapid change
# element=7
# data=specified_valid_vowel_list[element][780:1980]
# fig=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data))),
#     y=data,
#     ),
# )

# fig.update_layout(
#     template='plotly_white',
# )

# fig.show()
# point_Cloud=timedelay.TimeDelayEmbedding(M, 1, 1)
# Points=point_Cloud(data)
# print(len(Points))
# dgms = ripser(Points,maxdim=1)['dgms']
# dgms = dgms[1]
# x=[dgms[ele][0] for ele in range(len(dgms))]
# y=[dgms[ele][1]-dgms[ele][0] for ele in range(len(dgms))]

# print('maximal persistence is ', max(y))


# fig2=go.Figure(
#     data=go.Scatter(
#     x=x,
#     y=y,
#     mode='markers'
#     )
# )
# fig2.update_layout(
#     width=500,
#     template='plotly_white',
# )
# fig2.show()

# #pio.write_image(fig, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_rapid_series.pdf',scale=1,width=1000)

# fig3=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data[0:600]))),
#     y=data[0:600],
#     ),
# )

# fig3.update_layout(
#     template='plotly_white',
# )
# fig3.show()
# #pio.write_image(fig3, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_rapid_series1.pdf',scale=1,width=500)

# fig4=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data[0:800]))),
#     y=data[0:800],
#     ),
# )

# fig4.update_layout(
#     template='plotly_white',
# )
# fig4.show()
# #pio.write_image(fig4, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_rapid_series2.pdf',scale=1,width=650)

# fig5=go.Figure(
#     data=go.Scatter(
#     x=list(range(len(data[0:1000]))),
#     y=data[0:1000],
#     ),
# )

# fig5.update_layout(
#     template='plotly_white',
# )
# fig5.show()
# #pio.write_image(fig5, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_rapid_series3.pdf',scale=1,width=800)


In [6]:
x_end=1.2
y_end=.2
size=0.05
y_size=0.05


fig = make_subplots(
    rows=4, cols=2,
    column_widths=[0.5, 0.5],
    row_heights=[0.35,0.15,0.35,0.15],
    vertical_spacing=0.04,
    horizontal_spacing=0.1,
    specs=[[ {"type": "scatter"},{"type": "scatter"}],[ {"type": "Histogram2d"},{"type": "Histogram2d"}],[ {"type": "scatter"},{"type": "scatter"}],[ {"type": "Histogram2d"},{"type": "Histogram2d"}]])

fig.add_trace(
    go.Scatter(
    x=xm_1,
    y=ym_1,
    mode='markers',
    ),
    row=1,col=1,
)

fig.update_xaxes(
    range=[0,1.2],  # sets the range of xaxis
)

fig.add_trace(
    go.Histogram2d(
    x=xm_1,
    y=ym_1,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(
    x=xm_2,
    y=ym_2,
    mode='markers',
    ),
    row=1,col=2,
)

fig.add_trace(
    go.Histogram2d(
    x=xm_2,
    y=ym_2,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=2,
)

fig.add_trace(
    go.Scatter(
    x=xm_3,
    y=ym_3,
    mode='markers',
    ),
    row=3,col=1,
)

fig.add_trace(
    go.Histogram2d(
    x=xm_3,
    y=ym_3,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=1,
)

fig.add_trace(
    go.Scatter(
    x=xm,
    y=ym,
    mode='markers',
    ),
    row=3,col=2,
)

fig.add_trace(
    go.Histogram2d(
    x=xm,
    y=ym,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=2,
)

fig.update_layout(
    height=800,
    width=800,
    showlegend=False,
    template='simple_white'
)

y_range=1.1
fig['layout']['yaxis'].update(range=[0,y_range])
fig['layout']['yaxis2'].update(range=[0,y_range])
fig['layout']['yaxis5'].update(range=[0,y_range])
fig['layout']['yaxis6'].update(range=[0,y_range])

fig.show()

#pio.write_image(fig, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_mild.pdf',scale=1,height=800,width=800)


In [10]:
x_end=1.2
y_end=.2
size=0.05
y_size=0.05


fig = make_subplots(
    rows=4, cols=2,
    column_widths=[0.5, 0.5],
    row_heights=[0.35,0.15,0.35,0.15],
    vertical_spacing=0.04,
    horizontal_spacing=0.1,
    specs=[[ {"type": "scatter"},{"type": "scatter"}],[ {"type": "Histogram2d"},{"type": "Histogram2d"}],[ {"type": "scatter"},{"type": "scatter"}],[ {"type": "Histogram2d"},{"type": "Histogram2d"}]])

fig.add_trace(
    go.Scatter(
    x=x_1,
    y=y_1,
    mode='markers',
    ),
    row=1,col=1,
)

fig.update_xaxes(
    range=[0,1.2],  # sets the range of xaxis
)

fig.add_trace(
    go.Histogram2d(
    x=x_1,
    y=y_1,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(
    x=x_2,
    y=y_2,
    mode='markers',
    ),
    row=1,col=2,
)

fig.add_trace(
    go.Histogram2d(
    x=x_2,
    y=y_2,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=2,
)

fig.add_trace(
    go.Scatter(
    x=x_3,
    y=y_3,
    mode='markers',
    ),
    row=3,col=1,
)

fig.add_trace(
    go.Histogram2d(
    x=x_3,
    y=y_3,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=1,
)

fig.add_trace(
    go.Scatter(
    x=x,
    y=y,
    mode='markers',
    ),
    row=3,col=2,
)

fig.add_trace(
    go.Histogram2d(
    x=x,
    y=y,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=2,
)

fig.update_layout(
    height=800,
    width=800,
    showlegend=False,
    template='simple_white'
)

y_range=1.1
fig['layout']['yaxis'].update(range=[0,y_range])
fig['layout']['yaxis2'].update(range=[0,y_range])
fig['layout']['yaxis5'].update(range=[0,y_range])
fig['layout']['yaxis6'].update(range=[0,y_range])

fig.show()

#pio.write_image(fig, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_rapid.pdf',scale=1,height=800,width=800)


In [28]:
x_end=1.2
y_end=.2
size=0.05
y_size=0.05
marker_color=['#4d94ff','#53c653','#bf00ff','#ff8000']

fig = make_subplots(
    rows=4, cols=4,
    column_widths=[0.25, 0.25, 0.25,0.25],
    row_heights=[0.35,0.15,0.35,0.15],
    vertical_spacing=0.03,
    horizontal_spacing=0.045,
    specs=[[ {"type": "scatter"},{"type": "scatter"},{"type": "scatter"},{"type": "scatter"}],
           [ {"type": "Histogram2d"},{"type": "Histogram2d"},{"type": "Histogram2d"},{"type": "Histogram2d"}],
           [ {"type": "scatter"},{"type": "scatter"},{"type": "scatter"},{"type": "scatter"}],
           [ {"type": "Histogram2d"},{"type": "Histogram2d"},{"type": "Histogram2d"},{"type": "Histogram2d"}]])

fig.add_trace(
    go.Scatter(
    x=x_1,
    y=y_1,
    mode='markers',
    marker_color=marker_color[0],
    ),
    row=1,col=1,
)

fig.update_xaxes(
    range=[0,1.2],  # sets the range of xaxis
)

fig.add_trace(
    go.Histogram2d(
    x=x_1,
    y=y_1,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=1,
)

fig.add_trace(
    go.Scatter(
    x=x_2,
    y=y_2,
    mode='markers',
    marker_color=marker_color[1],
    ),
    row=1,col=2,
)

fig.add_trace(
    go.Histogram2d(
    x=x_2,
    y=y_2,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=2,
)

fig.add_trace(
    go.Scatter(
    x=x_3,
    y=y_3,
    mode='markers',
    marker_color=marker_color[2],
    ),
    row=3,col=1,
)

fig.add_trace(
    go.Histogram2d(
    x=x_3,
    y=y_3,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=1,
)

fig.add_trace(
    go.Scatter(
    x=x,
    y=y,
    mode='markers',
    marker_color=marker_color[3],
    ),
    row=3,col=2,
)

fig.add_trace(
    go.Histogram2d(
    x=x,
    y=y,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=2,
)
#mild-----------------------------------------------------------

fig.add_trace(
    go.Scatter(
    x=xm_1,
    y=ym_1,
    mode='markers',
    marker_color=marker_color[0],
    ),
    row=1,col=3,
)

fig.update_xaxes(
    range=[0,1.2],  # sets the range of xaxis
)

fig.add_trace(
    go.Histogram2d(
    x=xm_1,
    y=ym_1,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=3,
)

fig.add_trace(
    go.Scatter(
    x=xm_2,
    y=ym_2,
    mode='markers',
    marker_color=marker_color[1],
    ),
    row=1,col=4,
)

fig.add_trace(
    go.Histogram2d(
    x=xm_2,
    y=ym_2,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=2, col=4,
)

fig.add_trace(
    go.Scatter(
    x=xm_3,
    y=ym_3,
    mode='markers',
    marker_color=marker_color[2],
    ),
    row=3,col=3,
)

fig.add_trace(
    go.Histogram2d(
    x=xm_3,
    y=ym_3,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=3,
)

fig.add_trace(
    go.Scatter(
    x=xm,
    y=ym,
    mode='markers',
    marker_color=marker_color[3],
    ),
    row=3,col=4,
)

fig.add_trace(
    go.Histogram2d(
    x=xm,
    y=ym,
    autobinx=False,
    xbins=dict(start=0, end=x_end, size=size),
    autobiny=False,
    ybins=dict(start=0, end=y_end, size=y_size),
    coloraxis = "coloraxis",
    ),
    row=4, col=4,
)

fig.update_layout(
    height=600,
    width=1000,
    showlegend=False,
    template='simple_white'
)

y_range=1.1
fig['layout']['yaxis'].update(range=[0,y_range])
fig['layout']['yaxis2'].update(range=[0,y_range])
fig['layout']['yaxis3'].update(range=[0,y_range])
fig['layout']['yaxis4'].update(range=[0,y_range])
fig['layout']['yaxis9'].update(range=[0,y_range])
fig['layout']['yaxis10'].update(range=[0,y_range])
fig['layout']['yaxis11'].update(range=[0,y_range])
fig['layout']['yaxis12'].update(range=[0,y_range])


fig.show()

#pio.write_image(fig, 'E:/Topological_Analysis_of_Time_Series/pictures/pdf_graph_raw/Result_rapid2.pdf',scale=1,height=800,width=1200)

